In [1]:
# Importing Libs
import tweepy
import json
from kafka import KafkaConsumer, KafkaProducer
from tweepy import OAuthHandler

#Twitter Keys
consumer_key = "hBS2M9407C5vDTXodUMtfOTh9"
consumer_secret = "rh9UuGlqNhzHm6GYo0koWbJeaFVO0Ml72uBEcGd9rYS6ZSRIbO"
access_token = "1342336999-AjXWWz4qadipKXLwPxd6MyZsE8ZQyQvxgRJnf7H"
access_token_secret = "7xiPV4MjeqwINKsWd197bZ3DWeigc6Ez0hfoRP6jMhiTB"

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Creating the API object by passing in auth information
api = tweepy.API(auth)
#Connection Checking
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")
#kafka Producer    
producer = KafkaProducer(bootstrap_servers='192.168.99.100:9092')
topic_name = "tweet"

Authentication OK


In [2]:
#twitter Steam to store data in the form of Json
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('./home/tweets.json', 'a') as f:
                f.write(data)
                words = "iphoneXR"
                date_since = "2019-08-08"
                new_search = words + " -filter:retweets"
                tweets = tweepy.Cursor(api.search, q=new_search, lang="en", since=date_since).items(3)
                for tweet in tweets:
                    record = ''
                    record += str(tweet.user.id_str) #userId
                    record += ',' 
                    record += str(tweet.user.screen_name) #userscreen name
                    record += ','
                    record += str(tweet.user.followers_count) #followers count
                    record += ','
                    record += str(tweet.user.location) #location
                    record += ','
                    record += str(tweet.text) #tweet
                producer.send (topic_name,json.dumps(data).encode('utf-8'))
                return True
        except BaseException as e:
            #print(&quot;Error on_data: %s&quot; % str(e))
            return True
 
    def on_error(self, status):
        
        print(status)
        return True
    
try: 
    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(track=['#iphone'])
except KeyboardInterrupt:
    print("Streaming Feed Ended")

Streaming Feed Ended
